## json 데이터 로드 후 임베딩 및 chromaDB 저장

In [ ]:
import os
import json
import chromadb
from chromadb.config import Settings
from sentence_transformers import SentenceTransformer
from langchain.text_splitter import RecursiveCharacterTextSplitter

def embed_and_store_chromadb(json_path, chroma_path):
    """
    - json_path:  위에서 저장된 JSON 파일 경로
    - chroma_path: ChromaDB를 저장할 로컬 폴더 경로
    """

    # 1) ChromaDB 초기화 (PersistentClient)
    client = chromadb.PersistentClient(path=chroma_path)
    collection = client.get_or_create_collection(name="wanted_jobs")

    # 2) SentenceTransformer 모델 로드
    #  
    embedding_model = SentenceTransformer("nlpai-lab/KURE-v1")

    # 3) Text Splitter 설정
    #    chunk_size, chunk_overlap은 필요에 따라 조정
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=512,
        chunk_overlap=50
    )

    # 4) JSON 파일 읽기
    with open(json_path, "r", encoding="utf-8") as f:
        jobs = json.load(f)  # 리스트 형태

    # 5) 각 Job(공고)마다 임베딩 → ChromaDB 저장
    for idx, job in enumerate(jobs):
        position_name = job.get("PositionName", "N/A")
        experience = job.get("Experience", "N/A")

        # 섹션별 필드 정의 (필요하다면 더 많은 필드를 추가할 수도 있음)
        sections = ["MainTask", "Qualification", "Preferred"]

        for section in sections:
            raw_text = job.get(section, "").strip()
            if not raw_text:
                # 만약 해당 섹션이 비어 있으면 스킵
                continue

            # 1) 텍스트 청크 분할
            chunks = text_splitter.split_text(raw_text)

            # 2) 청크 임베딩
            embeddings = embedding_model.encode(chunks).tolist()

            # 3) ChromaDB에 저장
            for i, chunk in enumerate(chunks):
                doc_id = f"{idx}-{section}-{i}"  # ex) "0-MainTask-0"
                collection.add(
                    ids=[doc_id],
                    documents=[chunk],
                    embeddings=[embeddings[i]],
                    metadatas=[{
                        "PositionName": position_name,
                        "Experience": experience,
                        "Section": section  # "MainTask", "Qualification", "Preferred"
                    }]
                )

        print(f"✅ {idx+1}/{len(jobs)}번째 ({position_name}) 임베딩 & 저장 완료")

    print("\n모든 공고 임베딩 & ChromaDB 저장이 완료되었습니다.")

json_path = "./jobs.json"       # 크롤링 후 저장된 JSON 경로
chroma_db_path = "./chroma/chroma_db"  # ChromaDB 폴더 경로
    
embed_and_store_chromadb(json_path, chroma_db_path)

c:\Users\user\miniconda3\envs\job311\Lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\user\.cache\huggingface\hub\models--nlpai-lab--KURE-v1. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


✅ 1/3번째 (NLP AI 엔지니어) 임베딩 & 저장 완료
✅ 2/3번째 (AI 사업계획서/제안서 작성 담당자) 임베딩 & 저장 완료
✅ 3/3번째 (AI솔루션 영업직) 임베딩 & 저장 완료

모든 공고 임베딩 & ChromaDB 저장이 완료되었습니다.
